In [220]:
from PIL import Image
import pytesseract
import requests
from io import BytesIO

In [221]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [222]:
image_url = 'https://hips.hearstapps.com/hmg-prod/images/color-quotes44-1603314416.jpg?crop=0.96xw:1xh;center,top&resize=980:*'

# Download the image
response = requests.get(image_url)
image_data = BytesIO(response.content)
# Open the image using Pillow
image = Image.open(image_data)

# Display or process the image as needed
print(image)

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=980x1225 at 0x2C9680349A0>


In [223]:
# image_path = r'C:\Users\PARAM\Desktop\image.jpeg'
# image = Image.open('https://media.istockphoto.com/id/1002415624/photo/new-delhi-city-in-daytime.jpg?s=612x612&w=0&k=20&c=eZkR0veckRgWRSCkBEIINfVS_aElRO7i3sFOzH-ON-A=')
# print(image)

In [224]:
extracted_text = pytesseract.image_to_string(image)
# extracted_text = ' '.join(word for word in extracted_text.split() if word.isalpha())
print("Extracted Text:", extracted_text)

Extracted Text: ™ i

He squints towards the dancing shrapnel of
dying
light along a rooftop and he says | love
things only as they are
and I’m sure! once did too but | can’t prove
it to anyone these days
and he says the end isn’t always about what
dies and | know! know
or | knew once and now | write about
beautiful things
like I will never touch a beautiful thing again
and the man
looks me in the yes and he points to the
blue-orange vault
over heaven’s gates and he says the face of
everyone you miss
is up there and | know! know I can’t see
them but I know

Hanif
Willis-Abdurraqib




In [225]:
from googletrans import Translator

In [226]:
translator = Translator()

# Translate the extracted text to Hindi
translated_text = translator.translate(extracted_text, dest='hi').text

print("Translated Text:", translated_text)

Translated Text: ™ मैं

वह नाचने की छर्रे की ओर बढ़ता है
मरना
एक छत के साथ प्रकाश और वह कहता है |प्यार
चीजें केवल वैसा ही हैं
और मुझे यकीन है!एक बार भी किया लेकिन |साबित नहीं हो सकता
यह इन दिनों किसी को भी
और वह कहता है कि अंत हमेशा इस बारे में नहीं होता है
मर जाता है और |जानना!जानना
या |एक बार और अब पता था |के बारे में लिखो
सुंदर चीजें
जैसे मैं फिर से एक खूबसूरत चीज को कभी नहीं छूऊंगा
और आदमी
मुझे हाँ में दिखता है और वह इंगित करता है
नीला-नारंगी तिजोरी
स्वर्ग के द्वार पर और वह का चेहरा कहता है
हर कोई आपको याद करता है
वहाँ है और |जानना!पता है कि मैं नहीं देख सकता
उन्हें लेकिन मुझे पता है

हनीफ
विलिस-अबदुरराकिब


In [227]:
import cv2
import numpy as np
# avg_color='black'

In [228]:
def remove_text(input_image_path, output_image_path, text_region):
    # Load the image
    if input_image_path.startswith('http'):
        response = requests.get(input_image_path)
        image = cv2.imdecode(np.frombuffer(response.content, np.uint8), cv2.IMREAD_COLOR)
    else:
        image = cv2.imread(input_image_path)

    if image is None:
        raise ValueError("Failed to load the image. Check the input path.")

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Create a binary mask using adaptive thresholding
    _, mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)

    # Extract the region where the text is located
    x, y, w, h = text_region
    roi = mask[y:y+h, x:x+w]

    # Check if the ROI has a valid size
    if np.count_nonzero(roi) == 0:
        raise ValueError("Text region is empty.")

    # Ensure mask and image have the same size
    roi = cv2.resize(roi, (image.shape[1], image.shape[0]))
    
    # Inpainting to fill the text region with the surrounding background
    result = cv2.inpaint(image, mask, 3, cv2.INPAINT_TELEA)

    # Save the output image
    cv2.imwrite(output_image_path, result)

    average_color = cv2.mean(image, mask=roi)[:3]
    complementary_color_int = tuple(int(255 - component) for component in average_color)
    return complementary_color_int

# Example usage
input_image_path = image_url
output_image_path = r'C:\Users\PARAM\Desktop\img_without_txt.jpeg'

# Text region (specify the coordinates and dimensions of the text region)
text_region_coordinates = (100, 100, 200, 50)  # Example coordinates (x, y, width, height)

avg_color = remove_text(input_image_path, output_image_path, text_region_coordinates)
print("Text Color (BGR):", avg_color)

Text Color (BGR): (71, 76, 34)


In [229]:
from PIL import ImageDraw, ImageFont
image_o = Image.open(output_image_path)

In [230]:
draw = ImageDraw.Draw(image_o)

In [231]:
# def wrap_text(text, max_words_per_line):
#     words = text.split()
#     lines = []
#     current_line = []

#     for word in words:
#         current_line.append(word)
#         if len(current_line) == max_words_per_line:
#             lines.append(' '.join(current_line))
#             current_line = []

#     if current_line:
#         lines.append(' '.join(current_line))

#     return lines

In [232]:
font = ImageFont.truetype(r"C:\Users\PARAM\Downloads\Noto_Sans_Devanagari\NotoSansDevanagari-VariableFont_wdth,wght.ttf", size=40)

In [233]:
text_position = (30, 30)

In [234]:
# max_words_per_line = 3

# # Wrap the text into lines with the specified number of words
# wrapped_lines = wrap_text(translated_text, max_words_per_line)
# _, _, _, baseline = font.getbbox(' ')
# line_spacing_factor = 2
# word_spacing_factor = 1.2
# line_height = int(baseline * line_spacing_factor)
# word_spacing = int(font.getbbox(' ')[2] * word_spacing_factor)
# Draw the wrapped lines on the image
# line_height = font.getbbox(' ')[1]  # Height of a line in the specified font
# for line in wrapped_lines:
draw.text(text_position, translated_text, font=font, fill=avg_color)
    # text_position = (text_position[0], text_position[1] + line_height)
    # text_position = (text_position[0] + word_spacing, text_position[1])

In [235]:
image_o.save(r'C:\Users\PARAM\Desktop\translated.jpeg')